# Business Problem

- Can a machine learning project be realized for salary predictions of Malagasy developers who work in Madagascar?

# Dataset Story

- This dataset was originally taken from the facebook group of Malagasy Developers 'facedev'
- This contains just few observations but let's try if it will fit a good Machine Learning

**Variables**

- **Local:** The local of the company(Tana, Toliara, Mahajanga) or Remote
- **Titre:** Title of the post occupated
- **Exp:** The experience in year
- **Salary:** Our main target in Ariary per Month


# Roadmap

- **1. Import required libraries**
- **2. Loading the dataset**
- **3. Exploratory Data Analysis**
- **4. Analysis of categorical variables**
- **5. Correlation Analysis**
- **6. Examining the Logarithm of the depedent variable**
- **7. Outliers analysis**
- **8. Feature extraction**
- **9. Encoding**
- **10. Standardization Process**
- **11. Creating Model**
- **12. Hyperparameter optimization**
- **13. Final model predictions and comparison with true prices**

# 1. Import Required Libraries

In [1]:
# Import librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)

# 2. Load the dataset

In [2]:
df = pd.read_csv("data/Karama (réponses) - Réponses _cleaned_.csv")

In [3]:
df.head(8)

,Firenena Hitoerana,Province/Remote,Titre,Taona niasana,Karama,Unité,Fréquence
0,Belgique,Bruxelles,Consultant IT,8,48 000,€ (EUR),Taona
1,Belgique,Belgique,Senior Fullstack.NET,10,66 000,€ (EUR),Taona
2,Canada,Québec,Analyste programmeur junior,"1,5",54 000,C$ (CAD),Taona
3,Canada,Québec,Junior,2,40 000,C$ (CAD),Taona
4,Canada,Québec,Dev Java,2,80 000,C$ (CAD),Taona
5,Canada,Québec,Product Owner,3,22 000,C$ (CAD),Taona
6,Canada,Québec,Analyste développeur,"4,5",72 900,C$ (CAD),Taona
7,Canada,Québec,Dev backend,6,70 000,C$ (CAD),Taona
